In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.data import HistoricTradeDataHandler
from EventDriven.event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-13 10:33:54 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
thetadata_start = '2021-01-01'

In [360]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
key = 8
with open(f'../input/profitable_weights_{key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{key}.csv').iloc[:, 1:]
# ttrades__ = ttrades__[ttrades__.Ticker.isin(['AAPL', 'BAC'])]
ttrades__#.Ticker.unique()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,13.0,506.0,521.0,103.280218,98.082018,-67.576607,-0.050331,2021-01-05,2021-01-27,22 days,SBUX
1,18.0,506.0,539.0,242.064271,216.884420,-453.237318,-0.104021,2021-01-05,2021-02-23,49 days,TSLA
2,4.0,506.0,549.0,523.375413,507.309998,-64.261661,-0.030696,2021-01-05,2021-03-09,63 days,NFLX
3,7.0,506.0,620.0,181.844242,206.179993,170.350257,0.133827,2021-01-05,2021-06-18,164 days,CAT
4,35.0,506.0,621.0,76.707542,76.849998,4.985961,0.001857,2021-01-05,2021-06-21,167 days,MU
5,20.0,506.0,647.0,112.803439,113.940002,22.731275,0.010076,2021-01-05,2021-07-28,204 days,TSM
6,19.0,506.0,756.0,30.275595,44.520000,270.643702,0.470491,2021-01-05,2021-12-31,360 days,BAC
7,11.0,528.0,539.0,243.077801,228.874762,-156.233427,-0.058430,2021-02-05,2021-02-23,18 days,MSFT
8,2.0,530.0,756.0,104.290746,145.544006,82.506520,0.395560,2021-02-09,2021-12-31,325 days,GOOG
9,16.0,540.0,546.0,238.113832,201.114847,-591.983762,-0.155384,2021-02-24,2021-03-04,8 days,TSLA


In [361]:
trades_ = ttrades__.copy()
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,13.0,506.0,521.0,103.280218,98.082018,-67.576607,-0.050331,2021-01-05,2021-01-27,22 days,SBUX
1,18.0,506.0,539.0,242.064271,216.884420,-453.237318,-0.104021,2021-01-05,2021-02-23,49 days,TSLA
2,4.0,506.0,549.0,523.375413,507.309998,-64.261661,-0.030696,2021-01-05,2021-03-09,63 days,NFLX
3,7.0,506.0,620.0,181.844242,206.179993,170.350257,0.133827,2021-01-05,2021-06-18,164 days,CAT
4,35.0,506.0,621.0,76.707542,76.849998,4.985961,0.001857,2021-01-05,2021-06-21,167 days,MU
5,20.0,506.0,647.0,112.803439,113.940002,22.731275,0.010076,2021-01-05,2021-07-28,204 days,TSM
6,19.0,506.0,756.0,30.275595,44.520000,270.643702,0.470491,2021-01-05,2021-12-31,360 days,BAC
7,11.0,528.0,539.0,243.077801,228.874762,-156.233427,-0.058430,2021-02-05,2021-02-23,18 days,MSFT
8,2.0,530.0,756.0,104.290746,145.544006,82.506520,0.395560,2021-02-09,2021-12-31,325 days,GOOG
9,16.0,540.0,546.0,238.113832,201.114847,-591.983762,-0.155384,2021-02-24,2021-03-04,8 days,TSLA


In [362]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)
pd.Series(weights).sort_values(ascending=False)

NVDA    0.170757
AMD     0.134075
TSLA    0.101147
MSFT    0.063441
AMZN    0.062433
AAPL    0.061262
MU      0.060557
BA      0.055017
TSM     0.051328
NFLX    0.048425
COST    0.035000
SBUX    0.032052
CAT     0.030901
GLD     0.020607
HD      0.019955
BAC     0.013145
META    0.012558
GOOG    0.005000
UNG     0.005000
dtype: float64

In [310]:

from EventDriven.riskmanager import RiskManager, close_cache, spot_cache, chain_cache, oi_cache, LOOKBACKS, order_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.liquidity_threshold = 2
rm.OrderPicker.lookback = 10
rm.OrderPicker.data_availability_threshold = 0.15
date, tick = '2023-07-05', 'AVGO'
date, tick = '2024-08-13', 'TSM'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': .50,
   'dte': 210,
   'moneyness_width': 0.5},
  #  {'direction': 'short',
  # 'rel_strike': .60,
  # 'dte': 270,
  # 'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}



order =  rm.OrderPicker.get_order(tick, date, right, 2, order_settings)
order

{'result': 'SUCCESSFUL',
 'data': {'long': ['TSM20250321C280'],
  'trade_id': '&L:TSM20250321C280',
  'close': 1.515}}

In [367]:
max_cash = {}
cash = 20_000
for s, w in weights.items():
    if w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash

{'AMD': 2,
 'NVDA': 2,
 'TSLA': 2,
 'MU': 2,
 'AAPL': 2,
 'AMZN': 2,
 'MSFT': 2,
 'NFLX': 2,
 'TSM': 2,
 'BA': 2,
 'CAT': 2,
 'COST': 2,
 'SBUX': 2,
 'HD': 2,
 'META': 2,
 'GLD': 2,
 'BAC': 2,
 'GOOG': 0.5,
 'UNG': 0.5}

In [369]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
from EventDriven.execution import exec_cache
evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)

In [370]:
evb_backtest.portfolio.initial_capital

20000.0

In [378]:
w_map = {x: w  * 0.85 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 100
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .850,
   'dte': 300,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 300,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}

# evb_backtest.portfolio.order_settings = {'type': 'naked',
#  'specifics': [{'direction': 'long',
#    'rel_strike': .750,
#    'dte': 210,
#    'moneyness_width': 0.5},

# ],
#  'name': 'vertical_spread'}


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.executor.commission_rate
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.roll_map = 30
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price
evb_backtest.portfolio.roll_map
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price


{'AMD': 2,
 'NVDA': 2,
 'TSLA': 2,
 'MU': 2,
 'AAPL': 2,
 'AMZN': 2,
 'MSFT': 2,
 'NFLX': 2,
 'TSM': 2,
 'BA': 2,
 'CAT': 2,
 'COST': 2,
 'SBUX': 2,
 'HD': 2,
 'META': 2,
 'GLD': 2,
 'BAC': 2,
 'GOOG': 0.5,
 'UNG': 0.5}

In [379]:
for key  in max_cash:
    if max_cash[key]*100 > evb_backtest.portfolio.allocated_cash_map[key]:
        print(key, max_cash[key]*100, evb_backtest.portfolio.allocated_cash_map[key])

In [380]:
pd.options.display.max_rows = 15
pd.options.display.max_columns = 15
signals = evb_backtest.bars.signal_df
# series = signals.set_index('Date')['MSFT']
# series[series != 0]
signals

,Date,SBUX,TSLA,NFLX,CAT,MU,TSM,...,META,NVDA,COST,AMZN,UNG,GLD,AMD
0,2021-01-05,1,1,1,1,1,1,...,0,0,0,0,0,0,0
1,2021-01-06,0,0,0,0,0,0,...,0,0,0,0,0,0,0
2,2021-01-07,0,0,0,0,0,0,...,0,0,0,0,0,0,0
3,2021-01-08,0,0,0,0,0,0,...,0,0,0,0,0,0,0
4,2021-01-11,0,0,0,0,0,0,...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,2021-12-27,0,0,0,0,0,0,...,0,0,0,0,0,0,0
255,2021-12-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0
256,2021-12-29,0,0,0,0,0,0,...,0,0,0,0,0,0,0
257,2021-12-30,0,0,0,0,0,0,...,0,0,0,0,0,0,0


In [381]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

36.0

## Problems:

- Current Problem: SELL Signal and Buy Signal are put right next to each other in the queue. Whereas, it is meant to be Sell Signal -> Order Event -> Fill Event -> Buy Signal -> Order Event -> Fill Event

Solution:
- Use a tuple of action ```python['CLOSE', 'OPEN']```
- Put first action into queue and return ffunctionality to backtester. Backtester then handles all corresponding sequence.
- Do the same for action two.

In [382]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET
Processing event: SIGNAL
Buy Details
Position: {'long': ['SBUX20220121C105'], 'short': ['SBUX20220121C110'], 'trade_id': '&L:SBUX20220121C105&S:SBUX20220121C110', 'close': 2.0}, Date: 2021-01-05, Signal: SignalEvent type:LONG, symbol=SBUX, date:2021-01-05 00:00:00, Order Settings=None, signal_id:SBUX20210105LONG
Max Contract Price: 2, Cash at Hand: 4.904007096317339
Order Quantity 2 Cash at Hand 4.904007096317339 Close 2.0
Processing event: SIGNAL
Not generating order because:MAX_PRICE_TOO_LOW at 2, adjusted to 2.4 SignalEvent type:LONG, symbol=TSLA, date:2021-01-05 00:00:00, Order Settings=None, signal_id:TSLA20210105LONG 
Processing event: SIGNAL
Not generating order because:MAX_PRICE_TOO_LOW at 2, adjusted to 2.4 SignalEvent type:LONG, symbol=NFLX, date:2021-01-05 00:00:00, Order Settings=None, signal_id:NFLX20210105LONG 
Processing event: SIGNAL


In [336]:
idx = 1
buys = [x  for x in exec_cache['fill'].values() if x.direction == 'BUY']
buy1 = buys[idx]
print("Buy")
print(buy1.datetime,buy1.fill_cost/buy1.quantity, buy1.market_value)

sells = [x  for x in exec_cache['fill'].values() if x.direction == 'SELL']
sell1 = sells[idx]
print("Sell")
print(sell1.datetime,sell1.fill_cost/sell1.quantity, sell1.market_value)


Buy
2023-07-24 00:00:00 2.0524030902941046 2394.2592280052786
Sell
2023-12-26 00:00:00 2.52451874599075 2979.047007793141


In [337]:
pd.options.display.max_rows = 100
trades = evb_backtest.portfolio._trades.copy()#
len(trades[['Positions','Ticker','PnL', 'ReturnPct', 'EntryPrice', 'ExitPrice', 'EntryMarketValue', 'ExitMarketValue', 'EntryTime', 'ExitTime', 'EntryPrice', 'ExitPrice', 'Quantity', 'SignalID', 'Duration']].SignalID.unique())
# len(trades['SignalID'].unique())

25

In [343]:
for v in (evb_backtest.portfolio.unprocessed_signals):
    if 'reason' not in v:
        print(v)

{'type': 'SIGNAL', 'symbol': 'GLD', 'datetime': Timestamp('2023-01-05 00:00:00'), 'signal_type': 'LONG', 'signal_id': 'GLD20230105LONG', 'order_settings': None}
{'type': 'SIGNAL', 'symbol': 'HD', 'datetime': Timestamp('2023-07-31 00:00:00'), 'signal_type': 'LONG', 'signal_id': 'HD20230731LONG', 'order_settings': None}
{'type': 'SIGNAL', 'symbol': 'MU', 'datetime': Timestamp('2023-11-16 00:00:00'), 'signal_type': 'LONG', 'signal_id': 'MU20231116LONG', 'order_settings': None}
{'type': 'SIGNAL', 'symbol': 'GLD', 'datetime': Timestamp('2023-12-05 00:00:00'), 'signal_type': 'LONG', 'signal_id': 'GLD20231205LONG', 'order_settings': None}
{'type': 'SIGNAL', 'symbol': 'HD', 'datetime': Timestamp('2023-12-06 00:00:00'), 'signal_type': 'LONG', 'signal_id': 'HD20231205LONG', 'order_settings': None}
{'type': 'SIGNAL', 'symbol': 'TLT', 'datetime': Timestamp('2023-12-08 00:00:00'), 'signal_type': 'LONG', 'signal_id': 'TLT20231207LONG', 'order_settings': {'type': 'naked', 'specifics': [{'direction': 

In [339]:
evb_backtest.portfolio.plot_portfolio()

In [329]:
evb_backtest.portfolio.aggregate()

Start                                                    2023-01-04 00:00:00
End                                                      2024-01-02 00:00:00
Duration                                                   363 days 00:00:00
Exposure Time [%]                                                      100.0
Equity Final [$]                                                    42789.02
Equity Peak [$]                                                 46751.761019
Return [%]                                                        114.018342
Buy & Hold Return [%]                                              45.822408
CAGR [%]                                                          114.917442
Volatility Ann. [%]                                                 60.59373
Sharpe Ratio                                                        1.553101
Sortino Ratio                                                        2.71802
Skew                                                                2.046695

In [21]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

,,long,short,trade_id,close,quantity,market_value
datetime,symbol,,,,,,
2023-01-25,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,770.0,4,770.0
2023-01-26,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,844.0,4,844.0
2023-01-27,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,976.0,4,976.0
2023-01-30,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,688.0,4,688.0
2023-01-31,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,730.0,4,730.0
2023-02-01,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,956.0,4,956.0
2023-02-02,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1358.0,4,1358.0
2023-02-03,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1244.0,4,1244.0
2023-02-06,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1218.0,4,1218.0


In [22]:


""""
Why do these two return weird stuff after run?
    current_weighted_holdings
    current_positions

I can't reconcile the cost with the data (NVM, haha)

"""
# evb_backtest.portfolio.all_positions

'"\nWhy do these two return weird stuff after run?\n    current_weighted_holdings\n    current_positions\n\nI can\'t reconcile the cost with the data (NVM, haha)\n\n'

In [23]:
order_cache['2024-01-03']

KeyError: '2024-01-03'

## Extend for  get_port_stats
- Buy & Hold
- Dates
- Trades
- _strategy in Aggregate
- The function

In [214]:
evb_backtest.trades.to_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/output/profitable_trades_options_{key}.csv')